In [1]:
import json
from typing import Tuple, List, Dict, Any, Optional, Union
import ijson
import hashlib
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

In [2]:
def findClosestPoint(
    point: Tuple[float, float, float], pointsStr: str
) -> Tuple[int, int]:
    # point is a tuple of (x, y, z)
    # pointsStr is a string of a list of points seperated by @
    stringPoints = pointsStr.split("@")
    point = [float(value) for value in point]
    # cast each strring from 0.1,0.2,0.3 to [0.1, 0.2, 0.3]
    points = [list(map(float, point.split(","))) for point in stringPoints]
    # find closest match of point in points
    closest = min(points, key=lambda p: sum((a - b) ** 2 for a, b in zip(p, point)))
    # return index of closest point and total length of points
    return (points.index(closest), len(points))

In [3]:
def distanceToLastRoadPoint(
        point: Tuple[float, float, float], pointsStr: str
) -> float:
    stringPoints = pointsStr.split("@")
    point = [float(value) for value in point]
    points = [list(map(float, point.split(","))) for point in stringPoints]
    lastPoint = points[-1]
    # find index in last point closest to 0.56
    # round to 2 decomal places
    if round(lastPoint[1], 1) == 0.5:
        lastPoint[1], lastPoint[2] = lastPoint[2], lastPoint[1]

    return sum((a - b) ** 2 for a, b in zip(lastPoint, point))

In [ ]:
road1 = []
road2 = []
road3 = []
road4 = []
road5 = []
road6 = []
road7 = []
road8 = []
road9 = []

def first_derivative(x):
                    return [abs(x[i+1] - x[i]) for i in range(len(x)-1)]


def loadJsons(filenames):
    results: Dict[str, Dict[int, Any]] = {}
    roads: Dict[str, Any] = {}
    road_success_dict = {}

    df_dict = {}
    for filename in filenames:
        # iterate over each object in the json file
        with open(filename, "r") as f:
            objects = ijson.items(f, "item")
            for benchmarking_obj in objects:
                # access the scenario
                scenario = benchmarking_obj["scenario"]
                # acess the waypoints within scenario
                waypoints = scenario["waypoints"]
                # get a hash of the waypoints
                waypointsHash = hashlib.sha256(str(waypoints).encode()).hexdigest()
                # if the hash equial asjdn write the object in a list
                # if waypointsHash == "44160852c628732f811ea9907657f37de4d02894da958e5d858d669ff2578995":
                #    road1.append(benchmarking_obj)
                # elif waypointsHash == "be859fbf2b9946c8d4834d026e413df51dc9f70bd637bcd0dc45ec7e8e3926af":
                #    road2.append(benchmarking_obj)
                # elif waypointsHash == "0a246a82022e55cbd10c2d46db0e90f41111798ccb7179b3c9ef1f9d7e549882":
                #    road3.append(benchmarking_obj)
                # elif waypointsHash == "493f3da92ee367c09bf8647aab2fb671462701ef0397c3749dd93b18585e1aa1":
                #    road4.append(benchmarking_obj)
                # elif waypointsHash == "382e50f80e16e79861d84d5d102dae4b793c449941b48d4963560d13e4b0f053":
                #    road5.append(benchmarking_obj)
                # elif waypointsHash == "90a8e61085e4e9918cb2c35f1d751b76a3a4b9ba5ef677deaa441487827fd026":
                #    road8.append(benchmarking_obj)

                roads[waypointsHash] = waypoints

                # get the perturbation_function
                perturbation_function = scenario["perturbation_function"]
                # get the perturbation_scale
                perturbation_scale = scenario["perturbation_scale"]
                # get the boolean value for isSuccess
                isSuccess = benchmarking_obj["isSuccess"]
                # get the boolean value for isSuccess
                timeout = benchmarking_obj["timeout"]

                # get the average value of the float list in xte
                xte = benchmarking_obj["xte"]
                xteAvg = sum([abs(i) for i in xte]) / len(xte)
                xteAvg = float(xteAvg)
                max_xte = max([abs(i) for i in xte])
                max_xte = float(max_xte)

                derivative = first_derivative(benchmarking_obj["xte"])


                


                # get the last value in the pos list
                pos = benchmarking_obj["pos"]
                print(pos[-1])
                posLast = pos[-1]
                # swap the y and z values
                posLast[1], posLast[2] = posLast[2], posLast[1]

                # find the closest point to the last value in pos
                closestPoint, totalPoints = findClosestPoint(posLast, waypoints)

                quickness = (closestPoint / totalPoints)

                # get distance to last road point
                distance = distanceToLastRoadPoint(posLast, waypoints)

                isSuccess = isSuccess

                time=len(xte)/20

                # check if there is a key for the perturbation_function
                if perturbation_function not in results:
                    results[perturbation_function] = {}
                # check if there is a key for the perturbation_scale
                if perturbation_scale not in results[perturbation_function]:
                    results[perturbation_function][perturbation_scale] = {
                        "success": [],
                        "xte": [],
                        "quickness": [],
                        "roadHash": [],
                        "time": [],
                        "timeout": [],
                        "maxxte": [],
                        "derivative": [],
                        "max_derivative": [],
                    }
                # increment the success or failure
                results[perturbation_function][perturbation_scale]["success"].append(
                    isSuccess
                )

                # append the xte and quickness values
                results[perturbation_function][perturbation_scale]["xte"].append(xteAvg)
                results[perturbation_function][perturbation_scale]["maxxte"].append(max_xte)
                results[perturbation_function][perturbation_scale]["quickness"].append(
                    quickness
                )
                results[perturbation_function][perturbation_scale]["time"].append(
                    time
                )
                results[perturbation_function][perturbation_scale]["roadHash"].append(
                    waypointsHash
                )
                results[perturbation_function][perturbation_scale]["timeout"].append(
                    timeout
                )
                results[perturbation_function][perturbation_scale]["derivative"].append(
                    derivative
                )

                results[perturbation_function][perturbation_scale]["max_derivative"].append(
                    max(derivative)
                )

                # other index
                index = f"Scale_{perturbation_scale}_Success"
                index2 = f"Scale_{perturbation_scale}_XTE"
                index3 = f"Scale_{perturbation_scale}_Quickness"
                index4 = f"Scale_{perturbation_scale}_RoadHash"
                index5 = f"Scale_{perturbation_scale}_MaxXTE"
                index6 = f"Scale_{perturbation_scale}_Distance"
                index7 = f"Scale_{perturbation_scale}_Time"
                index8 = f"Scale_{perturbation_scale}_Timeout"
                index9 = f"Scale_{perturbation_scale}_XTEderivative"


                if perturbation_function not in df_dict:
                    df_dict[perturbation_function] = {}
                if index not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index] = [isSuccess]
                elif (len(df_dict[perturbation_function][index])<10):
                    df_dict[perturbation_function][index].append(isSuccess)
                    print(len(df_dict[perturbation_function][index]))
                if index2 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index2] = [xteAvg]
                elif (len(df_dict[perturbation_function][index2])<=10):
                    df_dict[perturbation_function][index2].append(xteAvg)
                if index3 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index3] = [quickness]
                elif (len(df_dict[perturbation_function][index3])<=10):
                    df_dict[perturbation_function][index3].append(quickness)
                if index4 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index4] = [waypointsHash]
                elif (len(df_dict[perturbation_function][index4])<=10):
                    df_dict[perturbation_function][index4].append(waypointsHash)
                if index5 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index5] = [max_xte]
                elif (len(df_dict[perturbation_function][index5])<=10):
                    df_dict[perturbation_function][index5].append(max_xte)
                if index6 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index6] = [distance]
                elif (len(df_dict[perturbation_function][index6])<=10):
                    df_dict[perturbation_function][index6].append(distance)
                if index7 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index7] = [time]
                elif (len(df_dict[perturbation_function][index7])<=10):
                    df_dict[perturbation_function][index7].append(time)
                if index8 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index8] = [timeout]
                elif (len(df_dict[perturbation_function][index8])<=10):
                    df_dict[perturbation_function][index8].append(timeout)
                if index9 not in df_dict[perturbation_function]:
                    df_dict[perturbation_function][index9] = [derivative]
                elif (len(df_dict[perturbation_function][index9])<=10):
                    df_dict[perturbation_function][index9].append(derivative)

                # check if there is a key for roadHash in roads
                if waypointsHash not in roads:
                    roads[waypointsHash] = waypoints
                if waypointsHash not in road_success_dict:
                    road_success_dict[
                        waypointsHash
                    ] = f"{perturbation_function}_{perturbation_scale}-"
                if isSuccess:
                    road_success_dict[
                        waypointsHash
                    ] += f"{perturbation_function}_{perturbation_scale}-"


    return results, roads, road_success_dict, df_dict

# type="donkey"
type="udacity"

if type=="donkey":
    results, roads, road_success_dict, df_dict = loadJsons(["donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road1_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road2_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road3_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road4_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road5_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road6_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road7_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road8_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road9_dave.json","donkey_benchmark_normal_perturbations_dave/donkey_benchmark_normal_perturbations_road10_dave.json"])
    # results, roads, road_success_dict, df_dict = loadJsons(['donkey_benchmark_baseline_VANILLA_DAVE2.json'])

else:
    results, roads, road_success_dict, df_dict = loadJsons(["Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_0_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_1_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_2_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_3_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_4_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_5_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_6_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_7_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_8_dave2_udaciy_100k.json","Udacity_Benchmark/benchmark_udacity_normal_perturbations_road_9_dave2_udaciy_100k.json"])
    # results, roads, road_success_dict, df_dict = loadJsons(["benchmark_udacity_baseline_dave2_udaciy_100k.json"])


In [ ]:
import json

# iterate over a dict
for key, value in results.items():
    # iterate over a dict
    for key2, value2 in value.items():
        # iterate over a dict
        for key3, value3 in value2.items():
            # print the key and value
            print(key, key2, key3, value3)
            print(len(value3))


def delete_entries(json_file, field, values):
    # Read the JSON file
    with open(json_file, "r") as f:
        data = json.load(f)

    # Delete the first and third entry from the list
    data = data[1:3] + data[4:]

    # Write the updated JSON data back to the file
    with open(json_file, "w") as f:
        json.dump(data, f)


def findEntries(json_file, values):
    # Read the JSON file
    with open(json_file, "r") as f:
        data = json.load(f)

    # iterate over the data
    counter = 0
    for entry in data:
        counter += 1
        # check if the entry has the field
        if "scenario" in entry and entry["scenario"]["waypoints"] in values:
            print("counter has value in field")

    # Write the updated JSON data back to the file
    with open(json_file, "w") as f:
        json.dump(data, f)

def findEntries(json_file, values):
    # Read the JSON file
    with open(json_file, "r") as f:
        data = json.load(f)

    # iterate over the data
    counter = 0
    for entry in data:
        counter += 1
        # check if the entry has the field
        if "scenario" in entry and entry["scenario"]["waypoints"] in values:
            print("counter has value in field")

    # Write the updated JSON data back to the file
    with open(json_file, "w") as f:
        json.dump(data, f)

In [ ]:
df = pd.DataFrame.from_dict(df_dict, orient="index").round(2)

# copy dataframe as backup
df_copy = df.copy()

scale_0_columns = [col for col in df.columns if "Scale_0" in col and not "RoadHash" in col]
scale_1_columns = [col for col in df.columns if "Scale_1" in col and not "RoadHash" in col]
scale_2_columns = [col for col in df.columns if "Scale_2" in col and not "RoadHash" in col]
scale_3_columns = [col for col in df.columns if "Scale_3" in col and not "RoadHash" in col]
scale_4_columns = [col for col in df.columns if "Scale_4" in col and not "RoadHash" in col]


df.drop(
    [
        "Scale_0_RoadHash",
        "Scale_1_RoadHash",
        "Scale_2_RoadHash",
        "Scale_3_RoadHash",
        "Scale_4_RoadHash",
    ],
    axis=1,
    inplace=True,
)

In [23]:


def first_derivative(x):
    return [x[i+1] - x[i] for i in range(len(x)-1)]

# # Apply the first derivative to each XTE scale column
df["Scale_0_XTEderivative"] = df["Scale_0_XTE"].apply(first_derivative)
df["Scale_1_XTEderivative"] = df["Scale_1_XTE"].apply(first_derivative)
df["Scale_2_XTEderivative"] = df["Scale_2_XTE"].apply(first_derivative)
df["Scale_3_XTEderivative"] = df["Scale_3_XTE"].apply(first_derivative)
df["Scale_4_XTEderivative"] = df["Scale_4_XTE"].apply(first_derivative)


df["Scale_0_MaxXTE"] = df["Scale_0_MaxXTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_1_MaxXTE"] = df["Scale_1_MaxXTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_2_MaxXTE"] = df["Scale_2_MaxXTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_3_MaxXTE"] = df["Scale_3_MaxXTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_4_MaxXTE"] = df["Scale_4_MaxXTE"].apply(lambda x: (abs(i) for i in x))

df["Scale_0_XTE"] = df["Scale_0_XTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_1_XTE"] = df["Scale_1_XTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_2_XTE"] = df["Scale_2_XTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_3_XTE"] = df["Scale_3_XTE"].apply(lambda x: (abs(i) for i in x))
df["Scale_4_XTE"] = df["Scale_4_XTE"].apply(lambda x: (abs(i) for i in x))


df["Scale_0_XTEderivative"] = df["Scale_0_XTEderivative"].apply(lambda x: (abs(i) for i in x))
df["Scale_1_XTEderivative"] = df["Scale_1_XTEderivative"].apply(lambda x: (abs(i) for i in x))
df["Scale_2_XTEderivative"] = df["Scale_2_XTEderivative"].apply(lambda x: (abs(i) for i in x))
df["Scale_3_XTEderivative"] = df["Scale_3_XTEderivative"].apply(lambda x: (abs(i) for i in x))
df["Scale_4_XTEderivative"] = df["Scale_4_XTEderivative"].apply(lambda x: (abs(i) for i in x))




# TODO
df["Scale_0_Timeout"] = df.apply(lambda row: [False if float(s)>0.93 else q for q, s in zip(row["Scale_0_Timeout"], row["Scale_0_Quickness"])], axis=1)
df["Scale_1_Timeout"] = df.apply(lambda row: [False if float(s)>0.93 else q for q, s in zip(row["Scale_1_Timeout"], row["Scale_1_Quickness"])], axis=1)
df["Scale_2_Timeout"] = df.apply(lambda row: [False if float(s)>0.93 else q for q, s in zip(row["Scale_2_Timeout"], row["Scale_2_Quickness"])], axis=1)
df["Scale_3_Timeout"] = df.apply(lambda row: [False if float(s)>0.93 else q for q, s in zip(row["Scale_3_Timeout"], row["Scale_3_Quickness"])], axis=1)
df["Scale_4_Timeout"] = df.apply(lambda row: [False if float(s)>0.93 else q for q, s in zip(row["Scale_4_Timeout"], row["Scale_4_Quickness"])], axis=1)
# TODO


df["Scale_0_Success"] = df.apply(lambda row: [False if s else q for q, s in zip(row["Scale_0_Success"], row["Scale_0_Timeout"])], axis=1)
df["Scale_1_Success"] = df.apply(lambda row: [False if s else q for q, s in zip(row["Scale_1_Success"], row["Scale_1_Timeout"])], axis=1)
df["Scale_2_Success"] = df.apply(lambda row: [False if s else q for q, s in zip(row["Scale_2_Success"], row["Scale_2_Timeout"])], axis=1)
df["Scale_3_Success"] = df.apply(lambda row: [False if s else q for q, s in zip(row["Scale_3_Success"], row["Scale_3_Timeout"])], axis=1)
df["Scale_4_Success"] = df.apply(lambda row: [False if s else q for q, s in zip(row["Scale_4_Success"], row["Scale_4_Timeout"])], axis=1)


# Adjust Quickness based on Success before calculating the mean
df["Scale_0_Quickness"] = df.apply(lambda row: [1 if s and not t else 1-q for q, s, t in zip(row["Scale_0_Quickness"], row["Scale_0_Success"], row["Scale_0_Timeout"])], axis=1)
df["Scale_1_Quickness"] = df.apply(lambda row: [1 if s and not t else 1-q for q, s, t in zip(row["Scale_1_Quickness"], row["Scale_1_Success"], row["Scale_1_Timeout"])], axis=1)
df["Scale_2_Quickness"] = df.apply(lambda row: [1 if s and not t else 1-q for q, s, t in zip(row["Scale_2_Quickness"], row["Scale_2_Success"], row["Scale_2_Timeout"])], axis=1)
df["Scale_3_Quickness"] = df.apply(lambda row: [1 if s and not t else 1-q for q, s, t in zip(row["Scale_3_Quickness"], row["Scale_3_Success"], row["Scale_3_Timeout"])], axis=1)
df["Scale_4_Quickness"] = df.apply(lambda row: [1 if s and not t else 1-q for q, s, t in zip(row["Scale_4_Quickness"], row["Scale_4_Success"], row["Scale_4_Timeout"])], axis=1)



# # Adjust Time based on Success and Timeout before calculating the mean
df["Scale_0_Time"] = df.apply(lambda row: [q if s or t else 0 for q, s, t in zip(row["Scale_0_Time"], row["Scale_0_Success"], row["Scale_0_Timeout"])], axis=1)
df["Scale_1_Time"] = df.apply(lambda row: [q if s or t else 0 for q, s, t in zip(row["Scale_1_Time"], row["Scale_1_Success"], row["Scale_1_Timeout"])], axis=1)
df["Scale_2_Time"] = df.apply(lambda row: [q if s or t else 0 for q, s, t in zip(row["Scale_2_Time"], row["Scale_2_Success"], row["Scale_2_Timeout"])], axis=1)
df["Scale_3_Time"] = df.apply(lambda row: [q if s or t else 0 for q, s, t in zip(row["Scale_3_Time"], row["Scale_3_Success"], row["Scale_3_Timeout"])], axis=1)
df["Scale_4_Time"] = df.apply(lambda row: [q if s or t else 0 for q, s, t in zip(row["Scale_4_Time"], row["Scale_4_Success"], row["Scale_4_Timeout"])], axis=1)

df["Scale_0_XTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_0_XTE"], row["Scale_0_Success"], row["Scale_0_Timeout"])], axis=1)
df["Scale_1_XTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_1_XTE"], row["Scale_1_Success"], row["Scale_1_Timeout"])], axis=1)
df["Scale_2_XTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_2_XTE"], row["Scale_2_Success"], row["Scale_2_Timeout"])], axis=1)
df["Scale_3_XTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_3_XTE"], row["Scale_3_Success"], row["Scale_3_Timeout"])], axis=1)
df["Scale_4_XTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_4_XTE"], row["Scale_4_Success"], row["Scale_4_Timeout"])], axis=1)

df["Scale_0_MaxXTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_0_MaxXTE"], row["Scale_0_Success"], row["Scale_0_Timeout"])], axis=1)
df["Scale_1_MaxXTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_1_MaxXTE"], row["Scale_1_Success"], row["Scale_1_Timeout"])], axis=1)
df["Scale_2_MaxXTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_2_MaxXTE"], row["Scale_2_Success"], row["Scale_2_Timeout"])], axis=1)
df["Scale_3_MaxXTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_3_MaxXTE"], row["Scale_3_Success"], row["Scale_3_Timeout"])], axis=1)
df["Scale_4_MaxXTE"] = df.apply(lambda row: [q if s or not t else 0. for q, s, t in zip(row["Scale_4_MaxXTE"], row["Scale_4_Success"], row["Scale_4_Timeout"])], axis=1)

df["Scale_0_XTEderivative"] = df.apply(lambda row: [float(q) if s or not t else 0. for q, s, t in zip(row["Scale_0_XTEderivative"], row["Scale_0_Success"], row["Scale_0_Timeout"])], axis=1)
df["Scale_1_XTEderivative"] = df.apply(lambda row: [float(q) if s or not t else 0. for q, s, t in zip(row["Scale_1_XTEderivative"], row["Scale_1_Success"], row["Scale_1_Timeout"])], axis=1)
df["Scale_2_XTEderivative"] = df.apply(lambda row: [float(q) if s or not t else 0. for q, s, t in zip(row["Scale_2_XTEderivative"], row["Scale_2_Success"], row["Scale_2_Timeout"])], axis=1)
df["Scale_3_XTEderivative"] = df.apply(lambda row: [float(q) if s or not t else 0. for q, s, t in zip(row["Scale_3_XTEderivative"], row["Scale_3_Success"], row["Scale_3_Timeout"])], axis=1)
df["Scale_4_XTEderivative"] = df.apply(lambda row: [float(q) if s or not t else 0. for q, s, t in zip(row["Scale_4_XTEderivative"], row["Scale_4_Success"], row["Scale_4_Timeout"])], axis=1)


df["Scale_0_Quickness"] = df["Scale_0_Quickness"].apply(lambda x: sum(x)*100 / len(x))
df["Scale_0_Quickness"] = df["Scale_0_Quickness"].map("{:.2f}".format)

df["Scale_1_Quickness"] = df["Scale_1_Quickness"].apply(lambda x: sum(x)*100 / len(x))
df["Scale_1_Quickness"] = df["Scale_1_Quickness"].map("{:.2f}".format)

df["Scale_2_Quickness"] = df["Scale_2_Quickness"].apply(lambda x: sum(x)*100 / len(x))
df["Scale_2_Quickness"] = df["Scale_2_Quickness"].map("{:.2f}".format)

df["Scale_3_Quickness"] = df["Scale_3_Quickness"].apply(lambda x: sum(x)*100 / len(x))
df["Scale_3_Quickness"] = df["Scale_3_Quickness"].map("{:.2f}".format)

df["Scale_4_Quickness"] = df["Scale_4_Quickness"].apply(lambda x: sum(x)*100 / len(x))
df["Scale_4_Quickness"] = df["Scale_4_Quickness"].map("{:.2f}".format)


# For the Success columns, calculating the percentage of true values (as before)
df["Scale_0_Success"] = df["Scale_0_Success"].apply(lambda x: sum(x))
df["Scale_1_Success"] = df["Scale_1_Success"].apply(lambda x: sum(x))
df["Scale_2_Success"] = df["Scale_2_Success"].apply(lambda x: sum(x))
df["Scale_3_Success"] = df["Scale_3_Success"].apply(lambda x: sum(x))
df["Scale_4_Success"] = df["Scale_4_Success"].apply(lambda x: sum(x))

df["Scale_0_Timeout"] = df["Scale_0_Timeout"].apply(lambda x: sum(x))
df["Scale_1_Timeout"] = df["Scale_1_Timeout"].apply(lambda x: sum(x))
df["Scale_2_Timeout"] = df["Scale_2_Timeout"].apply(lambda x: sum(x))
df["Scale_3_Timeout"] = df["Scale_3_Timeout"].apply(lambda x: sum(x))
df["Scale_4_Timeout"] = df["Scale_4_Timeout"].apply(lambda x: sum(x))

if type=="donkey":
        df["Scale_0_XTEderivative"] = df["Scale_0_XTEderivative"].apply(lambda x: max(x)/2.5)
        df["Scale_1_XTEderivative"] = df["Scale_1_XTEderivative"].apply(lambda x: max(x)/2.5)
        df["Scale_2_XTEderivative"] = df["Scale_2_XTEderivative"].apply(lambda x: max(x)/2.5)
        df["Scale_3_XTEderivative"] = df["Scale_3_XTEderivative"].apply(lambda x: max(x)/2.5)
        df["Scale_4_XTEderivative"] = df["Scale_4_XTEderivative"].apply(lambda x: max(x)/2.5)
else:
        df["Scale_0_XTEderivative"] = df["Scale_0_XTEderivative"].apply(lambda x: max(x)/4.5)
        df["Scale_1_XTEderivative"] = df["Scale_1_XTEderivative"].apply(lambda x: max(x)/4.5)
        df["Scale_2_XTEderivative"] = df["Scale_2_XTEderivative"].apply(lambda x: max(x)/4.5)
        df["Scale_3_XTEderivative"] = df["Scale_3_XTEderivative"].apply(lambda x: max(x)/4.5)
        df["Scale_4_XTEderivative"] = df["Scale_4_XTEderivative"].apply(lambda x: max(x)/4.5)




df["Scale_0_Time"] = df["Scale_0_Time"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_1_Time"] = df["Scale_1_Time"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_2_Time"] = df["Scale_2_Time"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_3_Time"] = df["Scale_3_Time"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_4_Time"] = df["Scale_4_Time"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)

df["Scale_0_XTE"] = df["Scale_0_XTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_1_XTE"] = df["Scale_1_XTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_2_XTE"] = df["Scale_2_XTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_3_XTE"] = df["Scale_3_XTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_4_XTE"] = df["Scale_4_XTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)

df["Scale_0_MaxXTE"] = df["Scale_0_MaxXTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_1_MaxXTE"] = df["Scale_1_MaxXTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_2_MaxXTE"] = df["Scale_2_MaxXTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_3_MaxXTE"] = df["Scale_3_MaxXTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)
df["Scale_4_MaxXTE"] = df["Scale_4_MaxXTE"].apply(lambda x: sum([i for i in x if i != 0]) / len([i for i in x if i != 0]) if any(i != 0 for i in x) else 0)



# Finally, rounding the entire dataframe to 2 decimal places
df = df.round(2)

# Displaying the updated dataframe
df.head()

,Scale_0_Success,Scale_0_XTE,Scale_0_Quickness,Scale_0_MaxXTE,Scale_0_Distance,Scale_0_Time,Scale_0_Timeout,Scale_0_XTEderivative,Scale_1_Success,Scale_1_XTE,...,Scale_3_Timeout,Scale_3_XTEderivative,Scale_4_Success,Scale_4_XTE,Scale_4_Quickness,Scale_4_MaxXTE,Scale_4_Distance,Scale_4_Time,Scale_4_Timeout,Scale_4_XTEderivative
dynamic_snow_filter,10,1.99,100.00,2.96,"[40.51412019000011, 35.34271095772501, 18.2336...",85.20,0,0.07,10,2.02,...,2,0.18,6,2.36,79.85,3.81,"[46.043610510000086, 21125.041126239987, 49.87...",116.92,2,0.56
snow_filter,8,1.59,83.06,3.01,"[39.080562370000024, 38.156865552874464, 41.45...",63.79,0,0.08,5,1.31,...,0,0.12,0,0.86,76.94,4.08,"[19264.174872740005, 7507.028343669046, 19641....",0.00,0,0.13
histogram_equalisation,7,1.93,86.42,3.12,"[24002.010292360002, 37.06539515000023, 41.892...",137.19,2,0.20,7,2.19,...,5,0.56,3,2.55,87.54,3.64,"[27705.04026531, 46.45514194, 45.0791652500000...",265.27,6,0.57
posterize_filter,9,1.93,91.04,3.01,"[35.54594357000008, 35.64966141440052, 40.9539...",98.82,1,0.21,9,1.92,...,1,0.47,5,2.37,77.69,3.83,"[42.10367875999998, 50.21577552035245, 5331.36...",136.22,1,0.43
static_snow_filter,10,2.03,100.00,3.13,"[38.474057090000045, 36.687764187681516, 41.51...",93.45,0,0.17,9,2.07,...,1,0.51,7,2.00,89.33,3.26,"[44.8215410900002, 42.57788347763431, 17936.84...",182.00,3,0.51


In [24]:
import pandas as pd

# Assuming df is already created with columns such as 'Scale_0_XTE', 'Scale_0_Quickness', 'Scale_0_Success', etc.

# Step 1: Create a backup of the original DataFrame
df_copy = df.copy()

# Step 2: Extract the scale-related columns
scale_0_columns = [col for col in df.columns if "Scale_0" in col and not "RoadHash" in col]
scale_1_columns = [col for col in df.columns if "Scale_1" in col and not "RoadHash" in col]
scale_2_columns = [col for col in df.columns if "Scale_2" in col and not "RoadHash" in col]
scale_3_columns = [col for col in df.columns if "Scale_3" in col and not "RoadHash" in col]
scale_4_columns = [col for col in df.columns if "Scale_4" in col and not "RoadHash" in col]

# Step 3: Create the new reshaped DataFrame
reshaped_df = pd.DataFrame()
# Add Perturbation names (these are the index in the original DataFrame, so we reset it)
reshaped_df["Perturbation"] = pd.concat([df.index.to_series()] * 5, ignore_index=True)

# Add Scale column
reshaped_df["Scale"] = ["Scale_0", "Scale_1", "Scale_2", "Scale_3", "Scale_4"] * len(df)

# Extract XTE, Quickness, and Success into separate columns
reshaped_df["XTE"] = pd.concat([df[f"Scale_{i}_XTE"] for i in range(5)], ignore_index=True)
reshaped_df["Quickness"] = pd.concat([df[f"Scale_{i}_Quickness"] for i in range(5)], ignore_index=True)
reshaped_df["Success"] = pd.concat([df[f"Scale_{i}_Success"] for i in range(5)], ignore_index=True)
reshaped_df["MaxXTE"] = pd.concat([df[f"Scale_{i}_MaxXTE"] for i in range(5)], ignore_index=True)
reshaped_df["XTEderivative"] = pd.concat([df[f"Scale_{i}_XTEderivative"] for i in range(5)], ignore_index=True)
reshaped_df["Time"] = pd.concat([df[f"Scale_{i}_Time"] for i in range(5)], ignore_index=True)
reshaped_df["Timeout"] = pd.concat([df[f"Scale_{i}_Timeout"] for i in range(5)], ignore_index=True)

# Format the columns to two decimal places and percentages for Success
reshaped_df["XTE"] = reshaped_df["XTE"].astype(float).map("{:.2f}".format)
reshaped_df["Quickness"] = reshaped_df["Quickness"].astype(float).map("{:.2f}".format)
reshaped_df["Success"] = reshaped_df["Success"].astype(float).map("{:.2f}".format)
reshaped_df["Timeout"] = reshaped_df["Timeout"].astype(float).map("{:.2f}".format)
reshaped_df["MaxXTE"] = reshaped_df["MaxXTE"].astype(float).map("{:.2f}".format)
reshaped_df["XTEderivative"] = reshaped_df["XTEderivative"].astype(float).map("{:.2f}".format)
reshaped_df["Time"] = reshaped_df["Time"].astype(float).map("{:.2f}".format)

In [31]:
names={
    'gaussian_noise': '1 A-I',
    'poisson_noise': '2 A-II',
    'impulse_noise': '3 A-III',
    'jpeg_filter': '4 A-IV',
    'speckle_noise_filter': '5 A-V',
    'defocus_blur': '6 B-I',
    'motion_blur': '7 B-II',
    # 'zoom_blur': 'DISCARD',
    'gaussian_blur': '8 B-IV',
    'low_pass_filter': '9 B-V',
    'frost_filter': '10 C-I',
    # 'glass_blur':'glass blur todo',
    'snow_filter': '11 C-II',
    'fog_filter': '12 C-III',
    'increase_brightness': '13 C-IV',
    'contrast': '14 C-V',
    'elastic': '15 D-I',
    'pixelate': '16 D-II',
    'sample_pairing_filter': '17 D-III',
    'sharpen_filter': '18 D-IV',
    'scale_image': '19 E-II',
    'translate_image': '20 E-III',
    # 'reflection_filter': '21 reflection discard',
    'splatter_mapping': '22 F-I',
    'dotted_lines_mapping': '23 F-II',
    'zigzag_mapping': '24 F-III',
    'canny_edges_mapping': '25 F-IV',
    'cutout_filter': '26 F-V',
    'false_color_filter': '27 G-I',
    'phase_scrambling': '28 G-II',
    'histogram_equalisation': '29 G-III',
    'white_balance_filter': '30 G-IV',
    'grayscale_filter': '31 G-V',
    'saturation_filter': '32 G-VI',
    'saturation_decrease_filter': '33 G-VI',
    'posterize_filter': '34 G-VII'
}

In [32]:
import pandas as pd
import numpy as np

df = reshaped_df.copy()

# Step 1: Convert XTE, Quickness, Time, Success, and Timeout to numeric
df['XTE'] = pd.to_numeric(df['XTE'], errors='coerce')
df['MaxXTE'] = pd.to_numeric(df['MaxXTE'], errors='coerce')  # Convert Timeout to numeric
df['XTEderivative'] = pd.to_numeric(df['XTEderivative'], errors='coerce')  # Convert Timeout to numeric
df['Quickness'] = pd.to_numeric(df['Quickness'], errors='coerce')  # Now used as completion rate
df['Time'] = pd.to_numeric(df['Time'], errors='coerce')  # Replacing Success with Time
df['Success'] = pd.to_numeric(df['Success'], errors='coerce')  # Convert Success to numeric
df['Timeout'] = pd.to_numeric(df['Timeout'], errors='coerce')  # Convert Timeout to numeric

# Step 2: Function to calculate failure type (T and O based on sum of Success and Timeout)
def calculate_failure_type(df):
    # Group by Perturbation and sum Success and Timeout
    grouped = df.groupby('Perturbation').agg(
        Success_Sum=('Success', 'sum'),
        Timeout_Sum=('Timeout', 'sum')
    ).reset_index()

    # Calculate O and T values
    grouped['O'] = 50-(grouped['Success_Sum'])-grouped['Timeout_Sum']
    grouped['T'] = grouped['Timeout_Sum']

    # Convert to string format: "O:O_Value/T:T_Value"
    grouped['failure_type'] = "O:" + grouped['O'].astype(int).astype(str) + "/T:" + grouped['T'].astype(int).astype(str)
    
    return grouped[['Perturbation', 'failure_type']]

# Add the new 'failure_type' column
failure_type_df = calculate_failure_type(df)

# Merge the failure type back into the summary DataFrame
df = df.merge(failure_type_df, on='Perturbation')

# Now that the failure type is added, proceed with the previous steps

# # Function to normalize values for specific metrics
def normalize_values(values, metric):
    if metric == 'XTE':
        # Normalize XTE where 0 maps to 0 and 4 maps to 1
        return [(min((val / 4.1),4)) for val in values]
    
    elif metric == 'Success':  # Now used as completion rate
        # Normalize Quickness where 0 maps to 0 and 1 maps to 1
        return values/10  # Since it's already between 0 and 1

    elif metric == 'Quickness':  # Now used as completion rate
        # Normalize Quickness where 0 maps to 0 and 1 maps to 1
        return values/100  # Since it's already between 0 and 1
    
    elif metric == 'Time':
        # Normalize Time, treating it as is (higher times are worse)
        max_time = 150
        return [val / max_time if val<150 else 1 for val in values]
    
    else:
        return values

# def normalize_values(values,dummy):
#     min_value = np.min(values)
#     max_value = np.max(values)
    
#     if min_value == max_value:
#         return [0.5 for _ in values]
    
#     return [(val - min_value) / (max_value - min_value) for val in values]

# Function to escape special LaTeX characters in Perturbation names
def escape_latex_special_chars(text):
    special_chars = {
        '&': r'\&',
        '%': r'\%',
        '$': r'\$',
        '#': r'\#',
        '_': r'\_',
        '{': r'\{',
        '}': r'\}',
        '~': r'\textasciitilde',
        '^': r'\textasciicircum',
        '\\': r'\textbackslash'
    }
    return ''.join(special_chars.get(c, c) for c in text)

# Step 3: Function to generate LaTeX \Chart{} command for each perturbation and metric
def generate_chart_latex(perturbation, metric, df):
    """Generate the LaTeX \Chart{} command for the given perturbation and metric across all scales."""
    scale_values = df[df['Perturbation'] == perturbation][metric].values
    
    # Normalize the values for the LaTeX barplot
    normalized_values = normalize_values(scale_values, metric)
    
    # Convert normalized values to LaTeX \Chart command with 5 bars
    chart_latex = f"\\Chart{{{normalized_values[0]:.2f}}}{{{normalized_values[1]:.2f}}}{{{normalized_values[2]:.2f}}}{{{normalized_values[3]:.2f}}}{{{normalized_values[4]:.2f}}}"
    
    return chart_latex

# Step 4: Group by Perturbation and calculate mean
df_summary = df.groupby('Perturbation').agg(
    XTE_Mean=('XTE', 'mean'),
    Quickness_Mean=('Quickness', 'mean'),  # This will be used as the completion rate
    Time_Mean=('Time', 'mean'),  # Use the Time column for average time
    Failure_Type=('failure_type', 'first')  # Get the failure_type
).reset_index()

# Step 5: Generate \Chart{} for each perturbation and metric
df_summary['Completion_Rate_Chart'] = df_summary['Perturbation'].apply(lambda p: generate_chart_latex(p, 'Quickness', df))  # Completion rate
df_summary['XTE_Chart'] = df_summary['Perturbation'].apply(lambda p: generate_chart_latex(p, 'XTE', df))
df_summary['Time_Chart'] = df_summary['Perturbation'].apply(lambda p: generate_chart_latex(p, 'Time', df))  # Time instead of Success

# Step 6: Create the LaTeX table with means, inline \Chart{}, and failure type
def dataframe_to_latex_with_charts(df):
    latex_lines = []
    latex_lines.append("\\begin{tabular}{lccccccc}")
    latex_lines.append("\\hline")
    latex_lines.append("Perturbation & \\multicolumn{2}{c}{Completion Rate} & \\multicolumn{2}{c}{XTE} & \\multicolumn{2}{c}{Time} & Failure Type \\\\")
    latex_lines.append(" & Mean & Trend & Mean & Trend & Mean & Trend & \\\\")
    latex_lines.append("\\hline")

    # Add rows for each perturbation
    for idx, row in df.iterrows():
        perturbation = row['Perturbation']
        
        # # Add row to the LaTeX table with the updated order of Completion Rate, XTE, Time, and Failure Type
        # latex_lines.append(f"{names[perturbation]} & {(row['Quickness_Mean']):.2f} & {row['Completion_Rate_Chart']} & "
        #                    f"{min(4.0,row['XTE_Mean']):.2f} & {row['XTE_Chart']} & "
        #                    f"{row['Time_Mean']:.2f} & {row['Time_Chart']} & "
        #                    f"{row['Failure_Type']} \\\\")
        
        # Add row to the LaTeX table with the updated order of Completion Rate, XTE, Time, and Failure Type
        latex_lines.append(f"{names[perturbation]} &            {row['Completion_Rate_Chart']} \\\\")
    
    latex_lines.append("\\hline")
    latex_lines.append("\\end{tabular}")

    return "\n".join(latex_lines)


order_array = list(names.keys())
# print(order_array)

# Set the custom order using pd.Categorical
df_summary['Perturbation'] = pd.Categorical(df_summary['Perturbation'], categories=order_array, ordered=True)

# Sort the DataFrame by the 'Perturbation' column using the custom order
df_sorted = df_summary.sort_values('Perturbation')

df_sorted.dropna(subset=['Perturbation'], inplace=True)

# Generate LaTeX table with inline barplots and failure type
latex_table = dataframe_to_latex_with_charts(df_sorted)

# Output the LaTeX table (you can write it to a file or print)
print(latex_table)

\begin{tabular}{lccccccc}
\hline
Perturbation & \multicolumn{2}{c}{Completion Rate} & \multicolumn{2}{c}{XTE} & \multicolumn{2}{c}{Time} & Failure Type \\
 & Mean & Trend & Mean & Trend & Mean & Trend & \\
\hline
1 A-I &            \Chart{1.00}{1.00}{1.00}{0.94}{0.94} \\
2 A-II &            \Chart{1.00}{1.00}{1.00}{1.00}{1.00} \\
3 A-III &            \Chart{1.00}{1.00}{0.92}{0.94}{1.00} \\
4 A-IV &            \Chart{1.00}{0.92}{1.00}{0.77}{0.94} \\
5 A-V &            \Chart{0.99}{0.99}{0.99}{0.99}{0.93} \\
6 B-I &            \Chart{1.00}{0.95}{1.00}{1.00}{0.95} \\
7 B-II &            \Chart{1.00}{0.95}{1.00}{1.00}{1.00} \\
8 B-IV &            \Chart{1.00}{0.95}{0.95}{0.95}{1.00} \\
9 B-V &            \Chart{0.93}{0.99}{0.99}{0.99}{0.99} \\
10 C-I &            \Chart{1.00}{1.00}{0.76}{0.86}{0.74} \\
11 C-II &            \Chart{0.83}{0.70}{0.72}{0.70}{0.77} \\
12 C-III &            \Chart{1.00}{1.00}{0.79}{0.79}{0.78} \\
13 C-IV &            \Chart{1.00}{1.00}{0.95}{0.92}{0.92} \\
14 C-V

In [33]:
# Step 4: Group by Perturbation and calculate mean and variance
df_summary = df.groupby('Perturbation').agg(
    XTE_Mean=('XTEderivative', 'mean'),
    XTE_Var=('XTEderivative', 'var'),
    Quickness_Mean=('Success', 'mean'),  # Completion rate mean
    Quickness_Var=('Success', 'var'),  # Completion rate variance
    Time_Mean=('Time', 'mean'),  # Time mean
    Time_Var=('Time', 'var'),  # Time variance
    Success_Sum=('Success', 'sum'),
    Timeout_Sum=('Timeout', 'sum')  # Use the sum of timeout and success for failure type
).reset_index()

# Calculate O and T values for failure type
df_summary['O'] = 50 - df_summary['Success_Sum']-df_summary['Timeout_Sum']
df_summary['T'] = df_summary['Timeout_Sum']

df_summary['Completion_Rate_Chart'] = df_summary['Perturbation'].apply(lambda p: generate_chart_latex(p, 'Success', df))  # Completion rate

# Step 5: Generate \Chart{} for each perturbation and metric (if needed, here omitted)
# You can add this if you need the charts for trends in columns like completion rate or driving quality

# Step 6: Create the LaTeX table with mean, variance, and failure type
def dataframe_to_latex_with_charts_v2(df):
    latex_lines = []
    latex_lines.append("\\begin{tabular}{lcccccccccccc}")
    latex_lines.append("\\hline")
    latex_lines.append("Perturbation & \\multicolumn{2}{c}{Completion Rate} & \\multicolumn{2}{c}{Failure Type} & \\multicolumn{2}{c}{Driving Quality (XTE)} & \\multicolumn{2}{c}{Time of Execution} \\\\")
    latex_lines.append(" & Mean & Variance & O & T & Mean & Variance & Mean & Variance \\\\")
    latex_lines.append("\\hline")

    # Add rows for each perturbation
    for idx, row in df.iterrows():
        if row['Perturbation'] not in ['candy','dynamic_lightning_filter','dynamic_object_overlay','dynamic_rain_filter','dynamic_smoke_filter','dynamic_snow_filter','dynamic_sun_filter','effects_attention_rain','effects_attention_rain_dynamic','effects_rain_dynamic','effects_snowflake_dynamic','feathers','la_muse','mosaic','static_lightning_filter','static_object_overlay','static_rain_filter','static_smoke_filter','static_snow_filter','static_sun_filter','the_scream','udnie','reflection_filter']:
            perturbation =names[row['Perturbation']]
            if type=="udacity":
                # Add row to the LaTeX table
                latex_lines.append(f"         & "
                                f"{(row['Quickness_Mean'])*10:.0f}\% & {row['Quickness_Var']*10:.2f} & {row['Completion_Rate_Chart']} & "  # Completion Rate: Mean and Variance
                                f"{int(row['O'])} & {int(row['T'])} & "  # Failure Type: O and T
                                f"{row['Time_Mean']:.1f} & {row['Time_Var']:.1f} & "
                                f"{min(1.0, (row['XTE_Mean']))*100:.1f}\% & {row['XTE_Var']*100:.2f} \\\\ ") 
                                # Time of Execution: Mean and Variance
            else:
                # Add row to the LaTeX table
                latex_lines.append(f"         & "
                                f"{(row['Quickness_Mean'])*10:.0f}\% & {row['Quickness_Var']*10:.2f} & {row['Completion_Rate_Chart']} &"  # Completion Rate: Mean and Variance
                                f"{int(row['O'])} & {int(row['T'])} & "  # Failure Type: O and T
                                f"{row['Time_Mean']:.1f} & {row['Time_Var']:.1f} & "
                                f"{min(1.0, (row['XTE_Mean']))*100:.1f}\% & {row['XTE_Var']*100:.2f} ") 
                                # Time of Execution: Mean and Variance
    
    latex_lines.append("\\hline")
    latex_lines.append("\\end{tabular}")

    return "\n".join(latex_lines)


order_array = list(names.keys())
# print(order_array)

# Set the custom order using pd.Categorical
df_summary['Perturbation'] = pd.Categorical(df_summary['Perturbation'], categories=order_array, ordered=True)

# Sort the DataFrame by the 'Perturbation' column using the custom order
df_sorted = df_summary.sort_values('Perturbation')

df_sorted.dropna(subset=['Perturbation'], inplace=True)
# print(df_sorted)

# Generate LaTeX table with the new structure
latex_table_v2 = dataframe_to_latex_with_charts_v2(df_sorted)

# Output the LaTeX table
print(latex_table_v2)

\begin{tabular}{lcccccccccccc}
\hline
Perturbation & \multicolumn{2}{c}{Completion Rate} & \multicolumn{2}{c}{Failure Type} & \multicolumn{2}{c}{Driving Quality (XTE)} & \multicolumn{2}{c}{Time of Execution} \\
 & Mean & Variance & O & T & Mean & Variance & Mean & Variance \\
\hline
         & 86\% & 3.00 & \Chart{0.90}{0.90}{0.90}{0.80}{0.80} & 2 & 5 & 95.6 & 39.2 & 44.8\% & 0.16 \\ 
         & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} & 0 & 0 & 50.2 & 3.2 & 9.4\% & 0.35 \\ 
         & 96\% & 3.00 & \Chart{1.00}{1.00}{0.90}{0.90}{1.00} & 0 & 2 & 129.9 & 51.9 & 11.0\% & 0.20 \\ 
         & 66\% & 88.00 & \Chart{1.00}{0.80}{0.80}{0.40}{0.30} & 1 & 16 & 149.2 & 3007.4 & 38.8\% & 3.06 \\ 
         & 88\% & 2.00 & \Chart{0.90}{0.90}{0.90}{0.90}{0.80} & 6 & 0 & 59.2 & 0.7 & 20.0\% & 0.04 \\ 
         & 96\% & 3.00 & \Chart{1.00}{0.90}{1.00}{1.00}{0.90} & 1 & 1 & 103.4 & 303.9 & 12.6\% & 0.15 \\ 
         & 84\% & 8.00 & \Chart{1.00}{0.80}{0.80}{0.80}{0.80} & 0 & 8 & 122.2 & 865.9 

In [ ]:
# A-I & 0.43 & 0.16 & 0.64 & 0.23                       & 98\% & 2.00 & \Chart{1.00}{1.00}{1.00}{1.00}{0.90} &0 & 1 & 83.3 & 229.9 & 9.4\% & 0.25                  & 86\% & 3.00 & \Chart{0.90}{0.90}{0.90}{0.80}{0.80} & 2 & 5 & 95.6 & 39.2 & 44.8\% & 0.16 \\ 
# A-II & 0.50 & 0.10 & 0.58 & 0.32                      & 98\% & 2.00 & \Chart{1.00}{1.00}{1.00}{1.00}{0.90} &1 & 0 & 52.2 & 3.5 & 9.0\% & 0.20                    & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} & 0 & 0 & 50.2 & 3.2 & 9.4\% & 0.35 \\ 
# A-III & 0.41 & 0.16 & 0.62 & 0.21                     & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} &0 & 0 & 95.5 & 130.1 & 7.6\% & 0.03                 & 96\% & 3.00 & \Chart{1.00}{1.00}{0.90}{0.90}{1.00} & 0 & 2 & 129.9 & 51.9 & 11.0\% & 0.20 \\ 
# A-IV & 0.59 & 0.02 & 0.62 & 0.56                      & 98\% & 2.00 & \Chart{0.90}{1.00}{1.00}{1.00}{1.00} &1 & 0 & 77.3 & 879.4 & 4.8\% & 0.06                  & 66\% & 88.00 & \Chart{1.00}{0.80}{0.80}{0.40}{0.30} & 1 & 16 & 149.2 & 3007.4 & 38.8\% & 3.06 \\ 
# A-V & 0.58 & 0.09 & 0.66 & 0.41                       & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} &0 & 0 & 73.4 & 5.3 & 6.2\% & 0.01                   & 88\% & 2.00 & \Chart{0.90}{0.90}{0.90}{0.90}{0.80} & 6 & 0 & 59.2 & 0.7 & 20.0\% & 0.04 \\ 
# B-I & 0.57 & 0.06 & 0.65 & 0.47                       & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} &0 & 0 & 84.6 & 18.1 & 8.0\% & 0.00                  & 96\% & 3.00 & \Chart{1.00}{0.90}{1.00}{1.00}{0.90} & 1 & 1 & 103.4 & 303.9 & 12.6\% & 0.15 \\ 
# B-II & 0.63 & 0.03 & 0.66 & 0.58                      & 94\% & 8.00 & \Chart{1.00}{0.80}{0.90}{1.00}{1.00} &3 & 0 & 93.8 & 39.2 & 8.6\% & 0.11                   & 84\% & 8.00 & \Chart{1.00}{0.80}{0.80}{0.80}{0.80} & 0 & 8 & 122.2 & 865.9 & 46.0\% & 3.06 \\ 
# B-IV & 0.58 & 0.08 & 0.66 & 0.46                      & 94\% & 3.00 & \Chart{1.00}{0.90}{0.90}{0.90}{1.00} &3 & 0 & 99.9 & 57.6 & 9.0\% & 0.18                   & 94\% & 3.00 & \Chart{1.00}{0.90}{0.90}{0.90}{1.00} & 3 & 0 & 87.6 & 100.6 & 11.8\% & 0.25 \\ 
# B-V & 0.63 & 0.01 & 0.64 & 0.62                       & 96\% & 3.00 & \Chart{1.00}{1.00}{1.00}{0.90}{0.90} &2 & 0 & 92.7 & 1.9 & 8.6\% & 0.07                    & 88\% & 2.00 & \Chart{0.80}{0.90}{0.90}{0.90}{0.90} & 6 & 0 & 88.0 & 12.0 & 28.6\% & 0.00 \\ 
# C-I & 0.62 & 0.04 & 0.65 & 0.54                       & 94\% & 3.00 & \Chart{0.90}{0.90}{0.90}{1.00}{1.00} &3 & 0 & 69.3 & 5.2 & 7.8\% & 0.03                    & 70\% & 135.00 & \Chart{1.00}{1.00}{0.70}{0.70}{0.10} & 15 & 0 & 52.1 & 94.3 & 11.6\% & 0.03 \\ 
# C-II & 0.64 & 0.04 & 0.67 & 0.56                      & 96\% & 8.00 & \Chart{1.00}{1.00}{1.00}{1.00}{0.80} &2 & 0 & 74.9 & 18.7 & 10.2\% & 0.15                  & 30\% & 120.00 & \Chart{0.80}{0.50}{0.20}{0.00}{0.00} & 35 & 0 & 32.5 & 924.2 & 13.8\% & 0.20 \\ 
# C-III & 0.56 & 0.14 & 0.67 & 0.29                     & 90\% & 5.00 & \Chart{0.90}{0.90}{1.00}{0.90}{0.80} &5 & 0 & 81.7 & 69.6 & 11.6\% & 0.35                  & 54\% & 228.00 & \Chart{1.00}{1.00}{0.60}{0.10}{0.00} & 23 & 0 & 52.2 & 1012.8 & 10.6\% & 0.14 \\ 
# C-IV & 0.66 & 0.00 & 0.66 & 0.65                      & 96\% & 8.00 & \Chart{1.00}{1.00}{1.00}{0.80}{1.00} &2 & 0 & 95.0 & 94.3 & 10.4\% & 0.15                  & 92\% & 7.00 & \Chart{1.00}{1.00}{0.90}{0.90}{0.80} & 3 & 1 & 123.2 & 235.4 & 12.2\% & 0.17 \\ 
# C-V & 0.66 & 0.00 & 0.66 & 0.65                       & 52\% & 197.00 & \Chart{1.00}{0.80}{0.70}{0.10}{0.00} &17 & 7 & 91.8 & 2971.1 & 13.8\% & 1.19             & 90\% & 20.00 & \Chart{1.00}{1.00}{1.00}{0.80}{0.70} & 4 & 1 & 92.5 & 322.6 & 16.4\% & 0.21 \\ 
# D-I & 0.63 & 0.01 & 0.64 & 0.61                       & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} &0 & 0 & 85.1 & 5.0 & 6.6\% & 0.16                   & 98\% & 2.00 & \Chart{1.00}{1.00}{1.00}{1.00}{0.90} & 0 & 1 & 73.3 & 154.3 & 9.6\% & 0.27 \\ 
# D-II & 0.62 & 0.05 & 0.66 & 0.53                      & 98\% & 2.00 & \Chart{1.00}{1.00}{0.90}{1.00}{1.00} &1 & 0 & 96.6 & 45.6 & 8.6\% & 0.19                   & 96\% & 8.00 & \Chart{1.00}{1.00}{0.80}{1.00}{1.00} & 2 & 0 & 84.4 & 7.2 & 14.4\% & 0.35 \\ 
# D-III & 0.41 & 0.16 & 0.64 & 0.20                     & 90\% & 15.00 & \Chart{1.00}{0.90}{0.90}{0.70}{1.00} &5 & 0 & 75.8 & 1392.2 & 15.2\% & 0.51               & 54\% & 213.00 & \Chart{1.00}{0.90}{0.70}{0.10}{0.00} & 21 & 2 & 86.6 & 2543.7 & 29.6\% & 3.92 \\ 
# D-IV & 0.66 & 0.01 & 0.67 & 0.64                      & 94\% & 8.00 & \Chart{1.00}{0.90}{1.00}{1.00}{0.80} &3 & 0 & 87.1 & 19.5 & 9.4\% & 0.10                   & 60\% & 255.00 & \Chart{1.00}{0.90}{1.00}{0.10}{0.00} & 20 & 0 & 64.0 & 1496.5 & 18.6\% & 0.74 \\ 
# E-II & 0.41 & 0.14 & 0.60 & 0.20                      & 46\% & 168.00 & \Chart{1.00}{0.20}{0.80}{0.20}{0.10} &15 & 12 & 123.7 & 918.6 & 15.0\% & 1.01            & 70\% & 45.00 & \Chart{0.90}{0.90}{0.60}{0.70}{0.40} & 5 & 10 & 220.7 & 3958.7 & 24.6\% & 0.71 \\ 
# E-III & 0.39 & 0.16 & 0.66 & 0.19                     & 64\% & 58.00 & \Chart{0.90}{0.90}{0.50}{0.40}{0.50} &18 & 0 & 74.4 & 342.8 & 11.4\% & 0.16               & 20\% & 120.00 & \Chart{0.80}{0.00}{0.20}{0.00}{0.00} & 29 & 11 & 323.8 & 14913.8 & 22.2\% & 0.41 \\ 
# F-I & 0.65 & 0.01 & 0.66 & 0.64                       & 66\% & 108.00 & \Chart{1.00}{1.00}{0.60}{0.40}{0.30} &14 & 3 & 90.3 & 210.5 & 11.8\% & 0.04              & 58\% & 122.00 & \Chart{0.80}{0.80}{0.80}{0.50}{0.00} & 16 & 5 & 156.8 & 8826.7 & 16.0\% & 0.14 \\ 
# F-II & 0.66 & 0.01 & 0.66 & 0.65                      & 80\% & 65.00 & \Chart{1.00}{1.00}{0.90}{0.70}{0.40} &5 & 5 & 108.2 & 264.1 & 8.2\% & 0.24                & 92\% & 7.00 & \Chart{1.00}{1.00}{0.90}{0.90}{0.80} & 3 & 1 & 92.6 & 237.6 & 18.6\% & 0.56 \\ 
# F-III & 0.66 & 0.00 & 0.66 & 0.65                     & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} &0 & 0 & 91.7 & 8.0 & 5.6\% & 0.03                   & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} & 0 & 0 & 77.2 & 9.7 & 17.4\% & 0.13 \\ 
# F-IV & 0.63 & 0.00 & 0.64 & 0.63                      & 100\% & 0.00 & \Chart{1.00}{1.00}{1.00}{1.00}{1.00} &0 & 0 & 85.3 & 0.5 & 7.2\% & 0.02                   & 88\% & 2.00 & \Chart{0.80}{0.90}{0.90}{0.90}{0.90} & 6 & 0 & 90.0 & 2.5 & 19.4\% & 0.01 \\ 
# F-V & 0.64 & 0.02 & 0.66 & 0.60                       & 66\% & 33.00 & \Chart{0.80}{0.90}{0.60}{0.50}{0.50} &15 & 2 & 99.8 & 175.3 & 14.6\% & 0.30               & 84\% & 43.00 & \Chart{1.00}{0.90}{1.00}{0.80}{0.50} & 8 & 0 & 123.4 & 731.7 & 12.2\% & 0.38 \\ 
# G-I & 0.50 & 0.09 & 0.58 & 0.34                       & 82\% & 92.00 & \Chart{0.30}{1.00}{1.00}{0.80}{1.00} &5 & 4 & 89.5 & 1336.1 & 12.4\% & 0.83               & 24\% & 153.00 & \Chart{0.00}{0.90}{0.00}{0.00}{0.30} & 38 & 0 & 40.8 & 3230.7 & 14.4\% & 0.57 \\ 
# G-II & 0.50 & 0.16 & 0.66 & 0.23                      & 46\% & 213.00 & \Chart{1.00}{0.90}{0.30}{0.10}{0.00} &21 & 6 & 49.0 & 986.5 & 15.2\% & 0.68              & 56\% & 113.00 & \Chart{0.90}{0.80}{0.70}{0.20}{0.20} & 19 & 3 & 44.3 & 78.9 & 33.4\% & 2.10 \\ 
# G-III & 0.65 & 0.01 & 0.66 & 0.64                     & 74\% & 28.00 & \Chart{1.00}{0.80}{0.70}{0.60}{0.60} &13 & 0 & 75.0 & 168.1 & 9.6\% & 0.25                & 52\% & 32.00 & \Chart{0.70}{0.70}{0.50}{0.40}{0.30} & 6 & 18 & 197.5 & 2127.2 & 49.0\% & 2.67 \\ 
# G-IV & 0.66 & 0.00 & 0.66 & 0.66                      & 92\% & 2.00 & \Chart{0.90}{0.90}{0.90}{1.00}{0.90} &4 & 0 & 92.2 & 32.5 & 8.0\% & 0.05                   & 94\% & 8.00 & \Chart{1.00}{1.00}{1.00}{0.90}{0.80} & 0 & 3 & 91.9 & 559.3 & 29.6\% & 3.32 \\ 
# G-V & 0.66 & 0.02 & 0.67 & 0.62                       & 88\% & 17.00 & \Chart{1.00}{1.00}{0.90}{0.80}{0.70} &6 & 0 & 85.5 & 75.8 & 11.8\% & 0.47                 & 66\% & 173.00 & \Chart{1.00}{0.90}{0.90}{0.50}{0.00} & 13 & 4 & 143.8 & 3136.5 & 32.0\% & 1.59 \\ 
# G-VI & 0.65 & 0.01 & 0.66 & 0.64                      & 46\% & 258.00 & \Chart{1.00}{1.00}{0.00}{0.30}{0.00} &0 & 27 & 131.5 & 932.6 & 6.2\% & 0.49              & 68\% & 72.00 & \Chart{0.90}{0.90}{0.80}{0.50}{0.30} & 11 & 5 & 118.8 & 2413.4 & 22.4\% & 1.09 \\ 
# G-VI & 0.64 & 0.03 & 0.66 & 0.60                      & 88\% & 7.00 & \Chart{0.90}{1.00}{0.80}{0.90}{0.80} &6 & 0 & 91.7 & 29.6 & 5.8\% & 0.01                   & 52\% & 187.00 & \Chart{0.90}{0.80}{0.80}{0.10}{0.00} & 15 & 9 & 103.6 & 123.9 & 17.6\% & 1.79 \\ 
# G-VII & 0.65 & 0.02 & 0.66 & 0.62                     & 86\% & 28.00 & \Chart{1.00}{1.00}{0.90}{0.80}{0.60} &7 & 0 & 78.8 & 297.7 & 14.6\% & 1.61                & 82\% & 32.00 & \Chart{0.90}{0.90}{0.90}{0.90}{0.50} & 4 & 5 & 108.1 & 259.0 & 40.2\% & 1.17 \\